Object Detection Evaluation through mAP(Mean Average Precision) Values

Completing this function for a project so I thought might as well make this function availabe for access to others for quick reference

In [3]:
import numpy as np
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
#GT Boxes
gt_boxes= {"img_00285.png": [[480, 457, 515, 529], [637, 435, 676, 536]]}
##why 2 boxes ---- keep 1
#Pred Boxes
pred_boxs={"img_00285.png": {"boxes": [[330, 463, 387, 505], [356, 456, 391, 521], [420, 433, 451, 498], [328, 465, 403, 540], [480, 477, 508, 522], [357, 460, 417, 537], [344, 459, 389, 493], [485, 459, 503, 511], [336, 463, 362, 496], [468, 435, 520, 521], [357, 458, 382, 485], [649, 479, 670, 531], [484, 455, 514, 519], [641, 439, 670, 532]], "scores": [0.0739, 0.0843, 0.091, 0.1008, 0.1012, 0.1058, 0.1243, 0.1266, 0.1342, 0.1618, 0.2452, 0.8505, 0.9113, 0.972]}}

In [5]:
def get_model_scores(pred_boxes):
    """Creates a dictionary of from model_scores to image ids.
    Args:
        pred_boxes (dict): dict of dicts of 'boxes' and 'scores'
    Returns:
        dict: keys are model_scores and values are image ids (usually filenames)
    """
    model_score={}
    for img_id, val in pred_boxes.items():
        for score in val['scores']:
            if score not in model_score.keys():
                model_score[score]=[img_id]
            else:
                model_score[score].append(img_id)
    return model_score

In [6]:
def calc_iou(gt_bbox, pred_bbox):
    '''
    This function takes the predicted bounding box and ground truth bounding box and 
    return the IoU ratio
    '''
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt= gt_bbox
    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p= pred_bbox
    
    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt> y_bottomright_gt):
        raise AssertionError("Ground Truth Bounding Box is not correct")
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p> y_bottomright_p):
        raise AssertionError("Predicted Bounding Box is not correct",x_topleft_p, x_bottomright_p,y_topleft_p,y_bottomright_gt)
        
         
    #if the GT bbox and predcited BBox do not overlap then iou=0
    if(x_bottomright_gt< x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox
        
        return 0.0
    if(y_bottomright_gt< y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox
        
        return 0.0
    if(x_topleft_gt> x_bottomright_p): # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox
        
        return 0.0
    if(y_topleft_gt> y_bottomright_p): # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox
        
        return 0.0
    
    
    GT_bbox_area = (x_bottomright_gt -  x_topleft_gt + 1) * (  y_bottomright_gt -y_topleft_gt + 1)
    Pred_bbox_area =(x_bottomright_p - x_topleft_p + 1 ) * ( y_bottomright_p -y_topleft_p + 1)
    
    # here we add +1 because range of pixels on a screen is inclusive:inclusive, So if we are at 3rd pixel then we are actually covering a length of 0 1 2 3 i.e 4 pixels
    
    x_top_left =np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])
    
    intersection_area = (x_bottom_right- x_top_left + 1) * (y_bottom_right-y_top_left  + 1)
    
    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)
   
    return intersection_area/union_area

In [7]:
'''
gt_bbox = list(gt_boxes.values())[0][0]
pred_boxes_cust = [[330, 463, 387, 505], [356, 456, 391, 521], [420, 433, 451, 498], [328, 465, 403, 540], [480, 477, 508, 522], [357, 460, 417, 537], [344, 459, 389, 493], [485, 459, 503, 511], [336, 463, 362, 496], [468, 435, 520, 521], [357, 458, 382, 485], [649, 479, 670, 531], [484, 455, 514, 519], [641, 439, 670, 532]]
len(pred_boxes_cust)

pred_box1= pred_boxes_cust[6]

for i, pred_box in enumerate(pred_boxs):
    print(i)
    print("----------")
    print(pred_box)

calc_iou(gt_bbox, pred_box1)
gt_boxes= {"img_00285.png": [[480, 457, 515, 529], [637, 435, 676, 536]]}
'''
def test_iou():
    pred_boxes_cust = [[330, 463, 387, 505], [356, 456, 391, 521], [420, 433, 451, 498], [328, 465, 403, 540], [480, 477, 508, 522], [357, 460, 417, 537], [344, 459, 389, 493], [485, 459, 503, 511], [336, 463, 362, 496], [468, 435, 520, 521], [357, 458, 382, 485], [649, 479, 670, 531], [484, 455, 514, 519], [641, 439, 670, 532]]
    ground_truth=[480,457,515,529]
    gt_boxs=[[480, 457, 515, 529], [637, 435, 676, 536]]
    for ipb, pred_box in enumerate(pred_boxes_cust):
        for igb,gt_box in enumerate(gt_boxs):
            iou= calc_iou(gt_box, pred_box)
            print("GTB=",gt_box,"\nPRED B=",pred_box)
            print("IOU=",iou)
            
test_iou()
get_model_scores(pred_boxs)

GTB= [480, 457, 515, 529] 
PRED B= [330, 463, 387, 505]
IOU= 0.0
GTB= [637, 435, 676, 536] 
PRED B= [330, 463, 387, 505]
IOU= 0.0
GTB= [480, 457, 515, 529] 
PRED B= [356, 456, 391, 521]
IOU= 0.0
GTB= [637, 435, 676, 536] 
PRED B= [356, 456, 391, 521]
IOU= 0.0
GTB= [480, 457, 515, 529] 
PRED B= [420, 433, 451, 498]
IOU= 0.0
GTB= [637, 435, 676, 536] 
PRED B= [420, 433, 451, 498]
IOU= 0.0
GTB= [480, 457, 515, 529] 
PRED B= [328, 465, 403, 540]
IOU= 0.0
GTB= [637, 435, 676, 536] 
PRED B= [328, 465, 403, 540]
IOU= 0.0
GTB= [480, 457, 515, 529] 
PRED B= [480, 477, 508, 522]
IOU= 0.5076103500761036
GTB= [637, 435, 676, 536] 
PRED B= [480, 477, 508, 522]
IOU= 0.0
GTB= [480, 457, 515, 529] 
PRED B= [357, 460, 417, 537]
IOU= 0.0
GTB= [637, 435, 676, 536] 
PRED B= [357, 460, 417, 537]
IOU= 0.0
GTB= [480, 457, 515, 529] 
PRED B= [344, 459, 389, 493]
IOU= 0.0
GTB= [637, 435, 676, 536] 
PRED B= [344, 459, 389, 493]
IOU= 0.0
GTB= [480, 457, 515, 529] 
PRED B= [485, 459, 503, 511]
IOU= 0.383181126331

{0.0739: ['img_00285.png'],
 0.0843: ['img_00285.png'],
 0.091: ['img_00285.png'],
 0.1008: ['img_00285.png'],
 0.1012: ['img_00285.png'],
 0.1058: ['img_00285.png'],
 0.1243: ['img_00285.png'],
 0.1266: ['img_00285.png'],
 0.1342: ['img_00285.png'],
 0.1618: ['img_00285.png'],
 0.2452: ['img_00285.png'],
 0.8505: ['img_00285.png'],
 0.9113: ['img_00285.png'],
 0.972: ['img_00285.png']}

In [8]:
def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.
    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.
    Returns:
        dict: true positives (int), false positives (int), false negatives (int)
    """
    all_pred_indices= range(len(pred_boxes))
    all_gt_indices=range(len(gt_boxes))
    if len(all_pred_indices)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    if len(all_gt_indices)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    
    gt_idx_thr=[]
    pred_idx_thr=[]
    ious=[]
    for ipb, pred_box in enumerate(pred_boxes):
        for igb, gt_box in enumerate(gt_boxes):
            iou= calc_iou(gt_box, pred_box)
            
            if iou >iou_thr:
                gt_idx_thr.append(igb)
                pred_idx_thr.append(ipb)
                ious.append(iou)
    iou_sort = np.argsort(ious)[::1]
    if len(iou_sort)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    else:
        gt_match_idx=[]
        pred_match_idx=[]
        for idx in iou_sort:
            gt_idx=gt_idx_thr[idx]
            pr_idx= pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if(gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp= len(gt_match_idx)
        fp= len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)
    return {'true_positive': tp, 'false_positive': fp, 'false_negative': fn}

In [9]:
pred_boxes_cust = [[330, 463, 387, 505], [356, 456, 391, 521], [420, 433, 451, 498], [328, 465, 403, 540], [480, 477, 508, 522], [357, 460, 417, 537], [344, 459, 389, 493], [485, 459, 503, 511], [336, 463, 362, 496], [468, 435, 520, 521], [357, 458, 382, 485], [649, 479, 670, 531], [484, 455, 514, 519], [641, 439, 670, 532]]
ground_truth=[480,457,515,529]

gt_boxs=[[480, 457, 515, 529], [637, 435, 676, 536]]

dict={}
dict=get_single_image_results(gt_boxs,pred_boxes_cust,0.5)
print(dict)

{'true_positive': 2, 'false_positive': 12, 'false_negative': 0}
